# **Importing Libraries**

In [ ]:
!pip install seqeval



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.5 MB/s eta 0:00:00

  Preparing metadata (setup.py) ... done








  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=9775a394d043f05b4d8a5bc9fbd764d7f2a333440665d7d8a0a8267e9349a48a

  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa

Successfully built seqeval




In [ ]:
import wandb
import os
command = 'huggingface-cli login --token "your_token_here"'
os.system(command)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.

Token is valid (permission: write).

Your token has been saved to /root/.cache/huggingface/token

Login successful


0

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertConfig, BertForTokenClassification
from seqeval.metrics import classification_report
from sklearn.metrics import accuracy_score
import copy

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
g1=pd.read_excel("/kaggle/input/miimasa/G1.xlsx")
g2=pd.read_excel("/kaggle/input/miimasa/G2.xlsx")
g3=pd.read_excel("/kaggle/input/miimasa/G3.xlsx")

g1_g2 = pd.concat([g1, g2, g3]).drop_duplicates()
all_g = pd.concat([g1, g2, g3]).drop_duplicates()

In [ ]:
g1.drop(["Unnamed: 0","ID"], axis=1, inplace=True)
g1.head()

,tags,text
0,"8:16:chronic_disease,20:32:treatment",portal fibrosis by liver biopsy
1,22:34:treatment,Contra-indication to liver biopsy
2,",32:44:treatment,,",Have a stable weight since the liver biopsy wa...
3,"26:38:treatment,",Subject agrees to have a liver biopsy performe...
4,",43:55:treatment,",Liver steatosis (on visual estimate or on live...


In [ ]:
g2.drop(["Unnamed: 0","ID"], axis=1, inplace=True)
g2.head()

,tags,text
0,"34:68:cancer,72:104:cancer",Participants with a diagnosis of chronic lymph...
1,"26:60:cancer,61:93:cancer",Histologically confirmed chronic lymphocytic l...
2,"7:16:chronic_disease,22:56:chronic_disease",Known infection with human immunodeficiency vi...
3,"21:30:chronic_disease,36:70:chronic_disease",Patients with known infection with human immun...
4,"15:49:chronic_disease,50:59:chronic_disease",Patients with human immunodeficiency virus (HI...


In [ ]:
g3.drop(["Unnamed: 0","ID"], axis=1, inplace=True)
g3.head()

,tags,text
0,"16:20:treatment,25:44:treatment",Current use of hemo- or peritoneal dialysis
1,"24:43:treatment,","Intention to change to peritoneal dialysis, or..."
2,"27:52:treatment,58:66:treatment",Participants with ongoing anticoagulation trea...
3,8:16:treatment,Use of warfarin
4,1:9:treatment,warfarin


In [ ]:
all_g.drop(["Unnamed: 0","ID"], axis=1, inplace=True)
all_g = all_g.reset_index(drop=True)

In [ ]:
g1_g2.drop(["Unnamed: 0","ID"], axis=1, inplace=True)
g1_g2 = g1_g2.reset_index(drop=True)

In [ ]:
# Representations of labels
# Using lowerlimit-1 is giving more sound results

print(g1.iloc[0,1][20-1:32])

liver biopsy


# **Preprocessing Functions**

The function below are helper functions which will assist in pre-processing datasets on the go.

In [ ]:
# Labels

entityset = [
    "treatment",
    "chronic_disease",
    "cancer",
    "gender",
    "pregnancy",
    "allergy_name",
    "contraception_consent",
    "language_literacy",
    "technology_access",
    "ethnicity",
    "attribute_clinical_variable",
    "age",
    "body_mass_index",
    "limit_upper_bound",
    "lower_bound"
]

In [ ]:
label_dict = {}

# Assigning O to 0
label_dict['O'] = 0

# Assigning unique values for B-entitytype and I-entitytype
for idx, label in enumerate(entityset, start=1):
    label_dict[f'B-{label}'] = idx
    label_dict[f'I-{label}'] = idx + len(entityset)

print(label_dict)

{'O': 0, 'B-treatment': 1, 'I-treatment': 16, 'B-chronic_disease': 2, 'I-chronic_disease': 17, 'B-cancer': 3, 'I-cancer': 18, 'B-gender': 4, 'I-gender': 19, 'B-pregnancy': 5, 'I-pregnancy': 20, 'B-allergy_name': 6, 'I-allergy_name': 21, 'B-contraception_consent': 7, 'I-contraception_consent': 22, 'B-language_literacy': 8, 'I-language_literacy': 23, 'B-technology_access': 9, 'I-technology_access': 24, 'B-ethnicity': 10, 'I-ethnicity': 25, 'B-attribute_clinical_variable': 11, 'I-attribute_clinical_variable': 26, 'B-age': 12, 'I-age': 27, 'B-body_mass_index': 13, 'I-body_mass_index': 28, 'B-limit_upper_bound': 14, 'I-limit_upper_bound': 29, 'B-lower_bound': 15, 'I-lower_bound': 30}


In [ ]:
label2id=label_dict
print(label2id)

id2label = {v: k for k, v in label2id.items()}
print(id2label)

{'O': 0, 'B-treatment': 1, 'I-treatment': 16, 'B-chronic_disease': 2, 'I-chronic_disease': 17, 'B-cancer': 3, 'I-cancer': 18, 'B-gender': 4, 'I-gender': 19, 'B-pregnancy': 5, 'I-pregnancy': 20, 'B-allergy_name': 6, 'I-allergy_name': 21, 'B-contraception_consent': 7, 'I-contraception_consent': 22, 'B-language_literacy': 8, 'I-language_literacy': 23, 'B-technology_access': 9, 'I-technology_access': 24, 'B-ethnicity': 10, 'I-ethnicity': 25, 'B-attribute_clinical_variable': 11, 'I-attribute_clinical_variable': 26, 'B-age': 12, 'I-age': 27, 'B-body_mass_index': 13, 'I-body_mass_index': 28, 'B-limit_upper_bound': 14, 'I-limit_upper_bound': 29, 'B-lower_bound': 15, 'I-lower_bound': 30}

{0: 'O', 1: 'B-treatment', 16: 'I-treatment', 2: 'B-chronic_disease', 17: 'I-chronic_disease', 3: 'B-cancer', 18: 'I-cancer', 4: 'B-gender', 19: 'I-gender', 5: 'B-pregnancy', 20: 'I-pregnancy', 6: 'B-allergy_name', 21: 'I-allergy_name', 7: 'B-contraception_consent', 22: 'I-contraception_consent', 8: 'B-languag

In [ ]:
def extract_words(row):
    tags = row['tags'].split(',')  # Split the tags by comma
    extracted_words = []
    if isinstance(row['text'], str)==False:
        return
    for tag in tags:
        tag_elements = tag.split(':')
        if len(tag_elements) != 3:
            # Skip this tag and continue with the next one
            continue
        start, end, label = tag_elements
        start = int(start)
        end = int(end)
        extracted_word = row['text'][start-1:end]  # Extract word based on start and end

        if extracted_word and not extracted_word[-1].isalpha():
            extracted_word = extracted_word[:-1]

        extracted_words.append({extracted_word:label})
    return extracted_words

In [ ]:
import re

# Function to generate text ids
def generate_text_ids(text):
    # Split text into words, including words separated by special characters
    if not isinstance(text, str):
        return []
    words = re.findall(r'\b\w+\b', text)
    return words

# Function to generate label ids
def generate_label_ids(labels, text_ids, entityset):
    if labels is None:
        return ['O'] * len(text_ids)  # Return default label list if labels is None

    label_ids = ['O'] * len(text_ids)
    for label_dict in labels:
        for entity, entity_type in label_dict.items():
            entity_words = re.findall(r'\b\w+\b', entity)
            entity_start_index = -1
            for i in range(len(text_ids) - len(entity_words) + 1):
                if text_ids[i:i+len(entity_words)] == entity_words:
                    entity_start_index = i
                    break
            if entity_start_index != -1:
                label_ids[entity_start_index] = 'B-' + entity_type
                for i in range(1, len(entity_words)):
                    label_ids[entity_start_index + i] = 'I-' + entity_type
    return label_ids



In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):
    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """

    tokenized_sentence = []
    labels = []

    sentence = sentence.strip()

    for word, label in zip(sentence.split(), text_labels.split(",")):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [ ]:
class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        # step 1: tokenize (and adapt corresponding labels)
        sentence = self.data.sentence[index]
        word_labels = self.data.word_labels[index]
        tokenized_sentence, labels = tokenize_and_preserve_labels(sentence, word_labels, self.tokenizer)

        # step 2: add special tokens (and corresponding labels)
        tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"] # add special tokens
        labels.insert(0, "O") # add outside label for [CLS] token
        labels.insert(-1, "O") # add outside label for [SEP] token

        # step 3: truncating/padding
        maxlen = self.max_len

        if (len(tokenized_sentence) > maxlen):
          # truncate
          tokenized_sentence = tokenized_sentence[:maxlen]
          labels = labels[:maxlen]
        else:
          # pad
          tokenized_sentence = tokenized_sentence + ['[PAD]'for _ in range(maxlen - len(tokenized_sentence))]
          labels = labels + ["O" for _ in range(maxlen - len(labels))]

        # step 4: obtain the attention mask
        attn_mask = [1 if tok != '[PAD]' else 0 for tok in tokenized_sentence]

        # step 5: convert tokens to input ids
        ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)

        label_ids = [label2id[label] for label in labels]
        # the following line is deprecated
        #label_ids = [label if label != 0 else -100 for label in label_ids]

        return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(attn_mask, dtype=torch.long),
              #'token_type_ids': torch.tensor(token_ids, dtype=torch.long),
              'targets': torch.tensor(label_ids, dtype=torch.long)
        }

    def __len__(self):
        return self.len

In [ ]:
# Defining a custom function to preprocess dataset according to our needs

In [ ]:
def preprocess_dataset(g1):
    # Apply the function to each row and save the result in a new column
    g1['labels'] = g1.apply(lambda row: extract_words(row), axis=1)
    #g1.drop('tags', axis=1, inplace=True)

    # Apply functions to DataFrame
    g1['text ids'] = g1['text'].apply(generate_text_ids)
    g1['label ids'] = g1.apply(lambda row: generate_label_ids(row['labels'], row['text ids'], entityset), axis=1)

    # Create a new column 'sentence' by joining the words in 'text_ids' with a space
    g1['sentence'] = g1['text ids'].apply(lambda x: ' '.join(x))

    # Create a new column 'word_labels' by converting the lists in 'text_ids' to comma-separated strings
    g1['word_labels'] = g1['label ids'].apply(lambda x: ','.join(x))

    g1.drop(["text", "labels", "text ids", "label ids", "tags"], axis=1, inplace=True)

    data = g1

    train_size = 0.8
    train_dataset = data.sample(frac=train_size, random_state=200)
    test_dataset = data.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    # Create keep_dataset by randomly selecting 100 samples from train_dataset
    keep_dataset = train_dataset.sample(n=100, random_state=42)
    keep_dataset = keep_dataset.reset_index(drop=True)

    print("FULL Dataset: {}".format(data.shape))
    print("TRAIN Dataset: {}".format(train_dataset.shape))
    print("TEST Dataset: {}".format(test_dataset.shape))
    print("KEEP Dataset: {}".format(keep_dataset.shape))

    training_set = dataset(train_dataset, tokenizer, MAX_LEN)
    testing_set = dataset(test_dataset, tokenizer, MAX_LEN)
    keep_set = dataset(keep_dataset, tokenizer, MAX_LEN)

    train_params = {'batch_size': TRAIN_BATCH_SIZE,
                    'shuffle': True,
                    'num_workers': 0
                    }

    test_params = {'batch_size': VALID_BATCH_SIZE,
                   'shuffle': True,
                   'num_workers': 0
                   }

    keep_params = {'batch_size': KEEP_BATCH_SIZE,
                   'shuffle': True,
                   'num_workers': 0
                   }

    training_loader = DataLoader(training_set, **train_params)
    testing_loader = DataLoader(testing_set, **test_params)
    keep_loader = DataLoader(keep_set, **keep_params)

    return training_set, testing_set, keep_set, training_loader, testing_loader, keep_loader


# **Loading Model**

In [ ]:
# Loading bert-base, A 110M parameter model, <500MB
# Everything that we achieve with this approach is good because this is the smallest popular transformer model

from torch import cuda

# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BertForTokenClassification.from_pretrained('bert-base-uncased',
                                                   num_labels=len(id2label),
                                                   id2label=id2label,
                                                   label2id=label2id)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model.to(device)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']

You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

# **Custom Train and Validation Functions**

In [ ]:
# # A custom training loop

def train(epoch, training_loader, optimizer):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()

    for idx, batch in enumerate(training_loader):

        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)

        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")

        # compute training accuracy
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_preds.extend(predictions)
        tr_labels.extend(targets)

        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [ ]:
# A custom validation loop

def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):

            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            targets = batch['targets'].to(device, dtype = torch.long)

            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits

            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)

            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")

            # compute evaluation accuracy
            flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
            active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            eval_labels.extend(targets)
            eval_preds.extend(predictions)

            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    #print(eval_labels)
    #print(eval_preds)

    labels = [id2label[id.item()] for id in eval_labels]
    predictions = [id2label[id.item()] for id in eval_preds]

    #print(labels)
    #print(predictions)

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

In [ ]:
# A custom function which can do the training just by taking as input the train set
# We do have a custom training function above but to call it, We need to go through all these steps
# This function and the one below is for easy access as we will do a lot of training and eval

def train_ner(training_set, training_loader):
    ids = training_set[0]["ids"].unsqueeze(0)
    mask = training_set[0]["mask"].unsqueeze(0)
    targets = training_set[0]["targets"].unsqueeze(0)

    ids = ids.to(device)
    mask = mask.to(device)
    targets = targets.to(device)

    outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
    initial_loss = outputs[0]
    initial_loss

    tr_logits = outputs[1]
    optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

    for epoch in range(EPOCHS):
        print(f"Training epoch: {epoch + 1}")
        train(epoch, training_loader, optimizer)

In [ ]:
# A custom function which can do the validation just by taking as input the test set

def val_ner(testing_loader):
    labels, predictions = valid(model, testing_loader)
    print(classification_report([labels], [predictions]))
    return(classification_report([labels], [predictions],output_dict=True))

# **Training on G1 for Task T1**

In [ ]:
#Training Args

MAX_LEN = 128
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 16
KEEP_BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10

In [ ]:
# Getting preprocessed dataset for task T1

training_set_g1, testing_set_g1, keep_set_g1, training_loader_g1, testing_loader_g1, keep_loader_g1 = preprocess_dataset(g1)

# Training

train_ner(training_set_g1, training_loader_g1)

# Validation

cls_rep=val_ner(testing_loader_g1)

FULL Dataset: (7356, 2)

TRAIN Dataset: (5885, 2)

TEST Dataset: (1471, 2)

KEEP Dataset: (100, 2)

Training epoch: 1

Training loss per 100 training steps: 3.6199393272399902

Training loss per 100 training steps: 0.6424711991359692

Training loss epoch: 0.4332884680074842

Training accuracy epoch: 0.6491089502333726

Training epoch: 2

Training loss per 100 training steps: 0.17029117047786713

Training loss per 100 training steps: 0.13962984630967132

Training loss epoch: 0.12812983094836058

Training accuracy epoch: 0.7544144427047156

Training epoch: 3

Training loss per 100 training steps: 0.11376184970140457

Training loss per 100 training steps: 0.09827355738028441

Training loss epoch: 0.09380341430559107

Training accuracy epoch: 0.8188104728831705

Training epoch: 4

Training loss per 100 training steps: 0.11574897170066833

Training loss per 100 training steps: 0.07571210859730693

Training loss epoch: 0.07321728561478465

Training accuracy epoch: 0.8601670845844663

Trainin

In [ ]:
# This is just for storing f1-score

clsrep_t123comb = pd.DataFrame(cls_rep).transpose()
# Convert dictionary to DataFrame
cls_rep_df = pd.DataFrame(cls_rep)
# Filter out rows containing "micro avg" and "macro avg"
filtered_df = cls_rep_df[~cls_rep_df.index.isin(['micro avg', 'macro avg'])]
# Extract the F1-score column
f1_scores = filtered_df.loc['f1-score']
# Optionally, convert the f1_scores to numeric if it's in string format
f1_scores_t1 = pd.to_numeric(f1_scores)

In [ ]:
model.push_to_hub("KaggleMasterX/BERT_Episode1")

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/KaggleMasterX/BERT_Episode1/commit/9588c30d3287b1a2b7d4be1ab6866938308b5779', commit_message='Upload BertForTokenClassification', commit_description='', oid='9588c30d3287b1a2b7d4be1ab6866938308b5779', pr_url=None, pr_revision=None, pr_num=None)

# **Custom Functions for EWC based Training**

In [ ]:
# Custom function for calculation of Fisher Information

def compute_fisher_information(training_loader, model, device):
    fisher_information = {}
    model.train()

    for batch in training_loader:
        ids = batch['ids'].to(device, dtype=torch.long)
        mask = batch['mask'].to(device, dtype=torch.long)
        targets = batch['targets'].to(device, dtype=torch.long)

        model.zero_grad()
        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss = outputs.loss
        loss.backward()

        # Compute squared gradients and accumulate
        for name, param in model.named_parameters():
            if param.requires_grad:
                if name not in fisher_information:
                    fisher_information[name] = (param.grad.detach() ** 2).clone()
                else:
                    fisher_information[name] += (param.grad.detach() ** 2).clone()

    return fisher_information


In [ ]:
# A custom function which calculates scale factor taking as input the fisher information of the parameters

def compute_ewc_lambda(fisher_information, scale_factor=1.0):
    """
    Compute the value of ewc_lambda (λ) based on the magnitude of Fisher Information.

    Args:
    - fisher_information (dict): Dictionary containing Fisher Information for each parameter.
    - scale_factor (float): Scaling factor to adjust the magnitude of λ. Default is 1.0.

    Returns:
    - ewc_lambda (float): Value of ewc_lambda (λ) that is inversely proportional to the magnitude of Fisher Information.
    """
    if not fisher_information:
        return 0.0

    total_fisher = sum(torch.sum(value) for value in fisher_information.values())
    if total_fisher == 0:
        return 0.0

    ewc_lambda = scale_factor / total_fisher
    return ewc_lambda


In [ ]:
# In elastic weight consolidation, We introduce a regularization term which constraints the parameter updates
# This constraint makes sure that Model doesn't forget previous knowledge
# I have made a custom training function for training the model using fisher information based constraints

def train_with_ewc(training_set, training_loader, ewc_lambda, fisher_information, device, model, optimizer):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()

    param_prior = {}  # Store the parameters from the previous task

    for name, param in model.named_parameters():
        param_prior[name] = param.detach().clone().cpu()

    for epoch in range(EPOCHS):
        print(f"Training epoch: {epoch + 1}")

        for idx, batch in enumerate(training_loader):
            ids = batch['ids'].to(device, dtype=torch.long)
            mask = batch['mask'].to(device, dtype=torch.long)
            targets = batch['targets'].to(device, dtype=torch.long)

            optimizer.zero_grad()
            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss = outputs.loss
            tr_loss += loss.item()

            nb_tr_steps += 1
            nb_tr_examples += targets.size(0)

            # compute training accuracy
            flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
            active_logits = outputs.logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
            active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            tr_preds.extend(predictions)
            tr_labels.extend(targets)

            tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            tr_accuracy += tmp_tr_accuracy

            # Compute EWC loss
            ewc_loss = 0
            if fisher_information:
                for name, param in model.named_parameters():
                    if name in fisher_information:
                        importance = fisher_information[name]
                        param_prior[name] = param_prior[name].to(device)  # Move param_prior to the same device as param
                        ewc_loss += (importance * (param - param_prior[name]) ** 2).sum()

            total_loss = loss + ewc_lambda * ewc_loss if ewc_loss != 0 else loss

            # gradient clipping
            torch.nn.utils.clip_grad_norm_(
                parameters=model.parameters(), max_norm=MAX_GRAD_NORM
            )

            # backward pass
            total_loss.backward()
            optimizer.step()

        epoch_loss = tr_loss / nb_tr_steps
        tr_accuracy = tr_accuracy / nb_tr_steps
        print(f"Training loss epoch: {epoch_loss}")
        print(f"Training accuracy epoch: {tr_accuracy}")

# **TASK 2**

In [ ]:
fisher_information = compute_fisher_information(training_loader_g1, model, device)

# Computing the Fisher Information for parameters of the Neural Network
# Represents the importance of paramters

In [ ]:
ewclambda=compute_ewc_lambda(fisher_information)

# EWC Lambda is a scaling term used in EWC method, It should be inversely proportional to the general trend of fisher infomration
# We have made a function above which calculates a value that follows the above description

In [ ]:
# Getting preprocessed dataset for Task T2

training_set_g2, testing_set_g2, keep_set_g2, training_loader_g2, testing_loader_g2, keep_loader_g2 = preprocess_dataset(g2)

FULL Dataset: (6455, 2)

TRAIN Dataset: (5164, 2)

TEST Dataset: (1291, 2)

KEEP Dataset: (100, 2)


In [ ]:
# Training our model on Task T1 while following a regularization powered by importance of weights of the model.

optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

train_with_ewc(training_set_g2, training_loader_g2, ewclambda, fisher_information, device, model, optimizer)


Training epoch: 1

Training loss epoch: 0.06507821040756909

Training accuracy epoch: 0.8692004453909362

Training epoch: 2

Training loss epoch: 0.05935637729909317

Training accuracy epoch: 0.4469961337790146

Training epoch: 3

Training loss epoch: 0.055069734120105275

Training accuracy epoch: 0.3020116179149848

Training epoch: 4

Training loss epoch: 0.05155102501013949

Training accuracy epoch: 0.22914892512443227

Training epoch: 5

Training loss epoch: 0.048381548347296535

Training accuracy epoch: 0.18567458734059983

Training epoch: 6

Training loss epoch: 0.045491128546710856

Training accuracy epoch: 0.15617945609673511

Training epoch: 7

Training loss epoch: 0.04285718960362294

Training accuracy epoch: 0.1348801282247596

Training epoch: 8

Training loss epoch: 0.040503083447994734

Training accuracy epoch: 0.11901899820698149

Training epoch: 9

Training loss epoch: 0.038322991471793834

Training accuracy epoch: 0.10649774022840826

Training epoch: 10

Training loss ep

In [ ]:
# Training the model on 100 kept samples from Task T1 [ Our model works well even without this, But this is one approach to continual learning ]

train_ner(keep_set_g1, keep_loader_g1)


Training epoch: 1

Training loss per 100 training steps: 0.026385173201560974

Training loss epoch: 0.031522838664906364

Training accuracy epoch: 0.9284241321444108

Training epoch: 2

Training loss per 100 training steps: 0.0281931571662426

Training loss epoch: 0.025137238004910096

Training accuracy epoch: 0.9451320178952501

Training epoch: 3

Training loss per 100 training steps: 0.01892845332622528

Training loss epoch: 0.018432808374719962

Training accuracy epoch: 0.9664601252006328

Training epoch: 4

Training loss per 100 training steps: 0.017087770625948906

Training loss epoch: 0.012774807401001453

Training accuracy epoch: 0.9760734224545874

Training epoch: 5

Training loss per 100 training steps: 0.012372449971735477

Training loss epoch: 0.009911395409809691

Training accuracy epoch: 0.98104411330375

Training epoch: 6

Training loss per 100 training steps: 0.007793717551976442

Training loss epoch: 0.010178630373307638

Training accuracy epoch: 0.9742457700157594

Tra

In [ ]:
# Preparing combined data for Tasks T1, T2 (We'll use this for validation in the next cell)

training_set_g1_g2, testing_set_g1_g2, keep_set_g1_g2, training_loader_g1_g2, testing_loader_g1_g2, keep_loader_g1_g2 = preprocess_dataset(g1_g2)

FULL Dataset: (20088, 2)

TRAIN Dataset: (16070, 2)

TEST Dataset: (4018, 2)

KEEP Dataset: (100, 2)


In [ ]:
# Validating the model on Tasks T1 and T2 after our continual learning approach

cls_rep=val_ner(testing_loader_g1_g2)

Validation loss per 100 evaluation steps: 0.03432496264576912

Validation loss per 100 evaluation steps: 0.059190237286067246

Validation loss per 100 evaluation steps: 0.058233328696470056

Validation Loss: 0.05790267709184379

Validation Accuracy: 0.916592252571287

                 precision    recall  f1-score   support



   allergy_name       0.83      0.82      0.82       711

         cancer       0.77      0.82      0.79      2712

chronic_disease       0.76      0.85      0.81      6781

      treatment       0.80      0.88      0.84      8836



      micro avg       0.78      0.86      0.82     19040

      macro avg       0.79      0.84      0.82     19040

   weighted avg       0.78      0.86      0.82     19040




In [ ]:
# This is just to store f1-score

clsrep_t123comb = pd.DataFrame(cls_rep).transpose()
# Convert dictionary to DataFrame
cls_rep_df = pd.DataFrame(cls_rep)
# Filter out rows containing "micro avg" and "macro avg"
filtered_df = cls_rep_df[~cls_rep_df.index.isin(['micro avg', 'macro avg'])]
# Extract the F1-score column
f1_scores = filtered_df.loc['f1-score']
# Optionally, convert the f1_scores to numeric if it's in string format
f1_scores_t1t2 = pd.to_numeric(f1_scores)

In [ ]:
model.push_to_hub("KaggleMasterX/BERT_Episode2")

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/KaggleMasterX/BERT_Episode2/commit/78727807647b2c84fe69bd1ad644dd5742e67393', commit_message='Upload BertForTokenClassification', commit_description='', oid='78727807647b2c84fe69bd1ad644dd5742e67393', pr_url=None, pr_revision=None, pr_num=None)

# **TASK 3**

In [ ]:
# Computing Fisher Information of Model paramters after 2 episodes, and computing a Lambda for the same

fisher_information = compute_fisher_information(training_loader_g2, model, device)

ewclambda=compute_ewc_lambda(fisher_information)


In [ ]:
# Preprocessing the data for Task T3

training_set_g3, testing_set_g3, keep_set_g3, training_loader_g3, testing_loader_g3, keep_loader_g3 = preprocess_dataset(g3)

# Training the current model on Task T3 now
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

train_with_ewc(training_set_g3, training_loader_g3, ewclambda, fisher_information, device, model, optimizer)

FULL Dataset: (6277, 2)

TRAIN Dataset: (5022, 2)

TEST Dataset: (1255, 2)

KEEP Dataset: (100, 2)

Training epoch: 1

Training loss epoch: 0.06058806278238631

Training accuracy epoch: 0.8802105538927782

Training epoch: 2

Training loss epoch: 0.05494608188486973

Training accuracy epoch: 0.45209191590695186

Training epoch: 3

Training loss epoch: 0.050767574016392356

Training accuracy epoch: 0.30503720005111934

Training epoch: 4

Training loss epoch: 0.04730835868116871

Training accuracy epoch: 0.23144904444461725

Training epoch: 5

Training loss epoch: 0.04430681694014247

Training accuracy epoch: 0.18707669053554568

Training epoch: 6

Training loss epoch: 0.04159930321603476

Training accuracy epoch: 0.1572810706834929

Training epoch: 7

Training loss epoch: 0.039092102478698126

Training accuracy epoch: 0.1360374772881113

Training epoch: 8

Training loss epoch: 0.03677321876774418

Training accuracy epoch: 0.11996813423902722

Training epoch: 9

Training loss epoch: 0.034

In [ ]:
# Training on just 100 preserved samples from T1 and T2 combined [ Our model performs well even without this but this is one approach to Continual Learning ]

train_ner(keep_set_g1, keep_loader_g1)
train_ner(keep_set_g2, keep_loader_g2)

Training epoch: 1

Training loss per 100 training steps: 0.034248173236846924

Training loss epoch: 0.02340352162718773

Training accuracy epoch: 0.9518195127875027

Training epoch: 2

Training loss per 100 training steps: 0.008711077272891998

Training loss epoch: 0.010410700725125415

Training accuracy epoch: 0.975549383527377

Training epoch: 3

Training loss per 100 training steps: 0.00996107142418623

Training loss epoch: 0.008298155053385667

Training accuracy epoch: 0.9807477355571758

Training epoch: 4

Training loss per 100 training steps: 0.007673389744013548

Training loss epoch: 0.005076557630673051

Training accuracy epoch: 0.9918812912790145

Training epoch: 5

Training loss per 100 training steps: 0.003785453736782074

Training loss epoch: 0.003921936033293605

Training accuracy epoch: 0.9938028907036103

Training epoch: 6

Training loss per 100 training steps: 0.004346095956861973

Training loss epoch: 0.003415538530264582

Training accuracy epoch: 0.995489435301393

Tr

In [ ]:
# Preparing the combined dataset for all tasks T1, T2 and T3

training_set_g, testing_set_g, keep_set_g, training_loader_g, testing_loader_g, keep_loader_g = preprocess_dataset(all_g)

FULL Dataset: (20088, 2)

TRAIN Dataset: (16070, 2)

TEST Dataset: (4018, 2)

KEEP Dataset: (100, 2)


In [ ]:
# Validating our continual learning based Model on all tasks T1, T2 and T3

cls_rep=val_ner(testing_loader_g)


Validation loss per 100 evaluation steps: 0.08108529448509216

Validation loss per 100 evaluation steps: 0.04385929467216466

Validation loss per 100 evaluation steps: 0.04478024677964233

Validation Loss: 0.04432936635497581

Validation Accuracy: 0.9366263113065599

                 precision    recall  f1-score   support



   allergy_name       0.90      0.90      0.90       711

         cancer       0.78      0.89      0.83      2712

chronic_disease       0.84      0.87      0.85      6781

      treatment       0.87      0.90      0.88      8836



      micro avg       0.85      0.89      0.87     19040

      macro avg       0.85      0.89      0.87     19040

   weighted avg       0.85      0.89      0.87     19040




In [ ]:
# This is just to fetch the f1-score

clsrep_t123comb = pd.DataFrame(cls_rep).transpose()
# Convert dictionary to DataFrame
cls_rep_df = pd.DataFrame(cls_rep)
# Filter out rows containing "micro avg" and "macro avg"
filtered_df = cls_rep_df[~cls_rep_df.index.isin(['micro avg', 'macro avg'])]
# Extract the F1-score column
f1_scores = filtered_df.loc['f1-score']
# Optionally, convert the f1_scores to numeric if it's in string format
f1_scores_t1t2t3 = pd.to_numeric(f1_scores)

In [ ]:
model.push_to_hub("KaggleMasterX/BERT_Episode3")

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/KaggleMasterX/BERT_Episode3/commit/8ebefb8835611b31105530a436501580d6fbc19f', commit_message='Upload BertForTokenClassification', commit_description='', oid='8ebefb8835611b31105530a436501580d6fbc19f', pr_url=None, pr_revision=None, pr_num=None)

# **Model 4 (For Testing)**

In [ ]:
#g4=pd.read_excel("/kaggle/input/miimasa/G4.xlsx")

In [ ]:
""""fisher_information = compute_fisher_information(training_loader_g3, model, device)

ewclambda=compute_ewc_lambda(fisher_information)


# Getting preprocessed dataset for a given dataframe

training_set_g4, testing_set_g4, keep_set_g4, training_loader_g4, testing_loader_g4, keep_loader_g4 = preprocess_dataset(g4)


# Training

optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

# Call the function
train_with_ewc(training_set_g4, training_loader_g4, ewclambda, fisher_information, device, model, optimizer)


# Validating the Updated Model on Tasks T1, T2, T3 after using those previous 100 examples

train_ner(keep_set_g, keep_loader_g)
val_ner(testing_loader_g)""""

Please use a file G4.csv to run the above 2 cells and it will be a test of our continual learning approach

# **A Model with G1, G2, G3 Combined**

In [ ]:
# Loading Model again to ensure it is pretrained model and not our fine-tuned model

from torch import cuda


# Check if CUDA is available and set the device

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BertForTokenClassification.from_pretrained('bert-base-uncased',
                                                   num_labels=len(id2label),
                                                   id2label=id2label,
                                                   label2id=label2id)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model.to(device)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']

You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [ ]:

# Fine-Tuning on Pre-Trained BERT-Base using all datasets together

train_ner(training_set_g, training_loader_g)

# Validation on the validation set produced out of all datasets collectively

val_ner(testing_loader_g)

Training epoch: 1

Training loss per 100 training steps: 3.5384016036987305

Training loss per 100 training steps: 0.6928099407122867

Training loss per 100 training steps: 0.4401657183223696

Training loss per 100 training steps: 0.34316632154078974

Training loss per 100 training steps: 0.2870647047003011

Training loss per 100 training steps: 0.25085872645209173

Training loss epoch: 0.2503981324242313

Training accuracy epoch: 0.7170884029660767

Training epoch: 2

Training loss per 100 training steps: 0.07849100232124329

Training loss per 100 training steps: 0.09188034326428234

Training loss per 100 training steps: 0.08638632642244225

Training loss per 100 training steps: 0.08298441354321483

Training loss per 100 training steps: 0.07989651686569996

Training loss per 100 training steps: 0.07768682895336085

Training loss epoch: 0.07755126204215745

Training accuracy epoch: 0.8533333538476084

Training epoch: 3

Training loss per 100 training steps: 0.061504095792770386

Traini

{'allergy_name': {'precision': 0.7994350282485876,
  'recall': 0.7960618846694796,
  'f1-score': 0.7977448907681467,
  'support': 711},
 'cancer': {'precision': 0.7352834958468761,
  'recall': 0.7507374631268436,
  'f1-score': 0.7429301222404671,
  'support': 2712},
 'chronic_disease': {'precision': 0.7712614155251142,
  'recall': 0.7970800766848547,
  'f1-score': 0.7839582275727028,
  'support': 6781},
 'treatment': {'precision': 0.7866013071895425,
  'recall': 0.8172249886826618,
  'f1-score': 0.801620781527531,
  'support': 8836},
 'micro avg': {'precision': 0.774370709382151,
  'recall': 0.7997899159663866,
  'f1-score': 0.7868750807389226,
  'support': 19040},
 'macro avg': {'precision': 0.7731453117025301,
  'recall': 0.79027610329096,
  'f1-score': 0.7815635055272119,
  'support': 19040},
 'weighted avg': {'precision': 0.7743077707365583,
  'recall': 0.7997899159663866,
  'f1-score': 0.7868258968277343,
  'support': 19040}}

In [ ]:
cls_rep=val_ner(testing_loader_g)

Validation loss per 100 evaluation steps: 0.07050216197967529

Validation loss per 100 evaluation steps: 0.06034811463502079

Validation loss per 100 evaluation steps: 0.06324942483664003

Validation Loss: 0.06429393810500937

Validation Accuracy: 0.8957069640996693

                 precision    recall  f1-score   support



   allergy_name       0.80      0.80      0.80       711

         cancer       0.74      0.75      0.74      2712

chronic_disease       0.77      0.80      0.78      6781

      treatment       0.79      0.82      0.80      8836



      micro avg       0.77      0.80      0.79     19040

      macro avg       0.77      0.79      0.78     19040

   weighted avg       0.77      0.80      0.79     19040




In [ ]:
# These are just to fetch f1-scores

clsrep_t123comb = pd.DataFrame(cls_rep).transpose()
# Convert dictionary to DataFrame
cls_rep_df = pd.DataFrame(cls_rep)
# Filter out rows containing "micro avg" and "macro avg"
filtered_df = cls_rep_df[~cls_rep_df.index.isin(['micro avg', 'macro avg'])]
# Extract the F1-score column
f1_scores = filtered_df.loc['f1-score']
# Optionally, convert the f1_scores to numeric if it's in string format
f1_scores_t1t2t3combined = pd.to_numeric(f1_scores)

In [ ]:
# Comparing performance | Continual Learning vs Learning all tasks together

series_list = [
    f1_scores_t1,
    f1_scores_t1t2,
    f1_scores_t1t2t3,
    f1_scores_t1t2t3combined
]

# Concatenate the Series
f1s = pd.concat(series_list, axis=1)

# Rename the columns
f1s.columns = [
'Performance on the test set of T1',
'Performance on the test set of T1 and T2',
'Performance on the test set of T1, T2 and T3',
'Performance on combined G1+G2+G3'

]

# Export to CSV
f1s.to_csv('f1s.csv', index=False)

f1s

,Performance on the test set of T1,Performance on the test set of T1 and T2,"Performance on the test set of T1, T2 and T3",Performance on combined G1+G2+G3
allergy_name,0.738386,0.824524,0.901130,0.797745
cancer,0.726179,0.793356,0.833593,0.742930
chronic_disease,0.779630,0.805484,0.854898,0.783958
treatment,0.777369,0.840783,0.881324,0.801621
micro avg,0.769029,0.820854,0.865570,0.786875
macro avg,0.755391,0.816037,0.867736,0.781564
weighted avg,0.768511,0.820849,0.865853,0.786826


In [ ]:
model.push_to_hub("KaggleMasterX/BERT_AllTasks")

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/KaggleMasterX/BERT_AllTasks/commit/e5d30676398f8ca06deb992fa44da65a45a2aeb5', commit_message='Upload BertForTokenClassification', commit_description='', oid='e5d30676398f8ca06deb992fa44da65a45a2aeb5', pr_url=None, pr_revision=None, pr_num=None)